In [27]:
import json
import pandas as pd
import re

def remove_empty_lines(file_path):
    with open(file_path, 'r', encoding='latin-1') as file:
        lines = file.readlines()

    # Remove empty lines
    lines = [line for line in lines if line.strip() != '']

    with open(file_path, 'w') as file:
        file.writelines(lines)

def create_comment_code_pairs(file_path):
    comment_code_pairs = {}
    current_comment = ''
    library_found = False
    exist = False
    actual = ''

    with open(file_path, 'r', encoding='latin-1') as file:
        for line in file:
            if 'library(' in line:
                exist = True
                break

        file.seek(0)  # Reset the file pointer to the beginning

        for line in file:
            #line = line.strip()

            if line.startswith('library('):
                # Found a library calling line
                library_found = True
            elif library_found or (not exist):
                if (not line.startswith((' ', '\t'))) and line.startswith('#'):
                    # Found a comment line after the last library calling
                    current_comment = str(current_comment) + str(line) + " "
                    actual = current_comment
                elif actual and (has_multiple_words(actual)):
                    # Found a code line following a comment
                    if (actual not in comment_code_pairs):
                        comment_code_pairs[actual] = ''

                    comment_code_pairs[actual]+= line + ' '
                    current_comment = ''
            # Reset the current comment if a new comment line is found
            #if (not line.startswith((' ', '\t'))) and line.startswith('#'):
                #print(line)
                #current_comment = str(current_comment) + str(line) + " "

    return comment_code_pairs

def remove_items_without_code(json_file_path):
    # Read the JSON file
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

    # Remove items without code
    data = [item for item in data if item.get('code') != ([""] and ["",""])]

    # Write the updated JSON file
    with open(json_file_path, 'w') as json_file:
        json.dump(data, json_file, indent=2)

def has_multiple_words(text):
    words = re.findall(r'\b[a-zA-Z]+\b', text)
    return len(words) > 1

In [ ]:
from tqdm import tqdm
dict = {'idx':[],
        'comment':[],
        'code':[]
       }

df = pd.DataFrame(dict)
json_file = open('output_file', 'a')
import os

directory_path = '/content/drive/MyDrive/CodeInspector/OSF_R_files'  # Replace with the path to your directory

# Get a list of all files in the directory
file_list = os.listdir(directory_path)
# Iterate over the files and read them
for file_name in tqdm(file_list):
      if file_name.endswith('.R') or file_name.endswith('.r'):

        #print(file_name)
        file_path = os.path.join(directory_path, file_name)
        try:
          remove_empty_lines(file_path)
        except:
          pass
        pairs = create_comment_code_pairs(file_path)
        comment_code_pairs = []
        # Print the comment-code pairs
        if pairs:
          j=0
          for comment, code_lines in pairs.items():
                                    idx = file_path.split('/')[-1] + ' ' + str(j)
                                    df.loc[len(df.index)] = [idx, comment, code_lines]
                                    j+=1


        #json.dump(comment_code_pairs, json_file, indent=2)
        #with open(output_file, 'a') as json_file:

In [ ]:
"""
# Json format
if pairs:
   for comment, code_lines in pairs.items():
                                    comment_code_pairs.append({
                                      "idx": f"{file_path.split('/')[-1]} { len(comment_code_pairs)+1}",
                                      "comment": comment,
                                      "code": code_lines
                                  })
    json.dump(comment_code_pairs, json_file, indent=2)

In [ ]:
df.head(30)

In [26]:
df.to_csv('/content/drive/MyDrive/CodeInspector/Data/Comment-code pairs/code-comment-pairs-m-words.csv', index=False, escapechar=';')